In [1]:
import numpy as np

In [2]:
def initialize(dict_network, seed = 99):
    np.random.seed(seed)
    
    for dict_layer in dict_network:        
        dim_layer_in = dict_layer['dim_in']
        dim_layer_out = dict_layer['dim_out']  
        dict_layer['W'] = np.random.uniform(-1, 1, (dim_layer_out, dim_layer_in))
        dict_layer['B'] = np.random.uniform(-1, 1, (dim_layer_out, 1))
        
    return dict_network

In [3]:
def forward_propagation(input_, dict_network):
    Z = input_
    
    for dict_layer in dict_network:

        activation = dict_layer['activation']
        activation_params = dict_layer['activation_params']
        W = dict_layer['W']
        B = dict_layer['B'].T.flatten()
    
        X = Z
        dict_layer['X'] = X
        
        V = np.dot(W, X) + B

        if len(activation_params) == 0:
            raise Exception('Activation parameter is empty.')   
            
        elif activation == 'logi':
            a = activation_params[0]
            Z = 1 / (1 + np.exp(-a*V))
            
        elif activation == 'tanh':
            a = activation_params[0]
            c = activation_params[1]
            Z = a * np.tanh(c*V)
            
        elif activation == 'relu':
            a = activation_params[0]
            Z = np.maximum(V,a*V)
            
        else:
            raise Exception('Activation function is not supported')
            
        dict_layer['Z'] = Z
        dict_layer['V'] = V

    return dict_network

In [4]:
def backward_propagation(output_data, dict_network, learning_rate=0.85):
    
    d = output_data    
    o = dict_network[-1]['Z']
    error = d - o
    
    for index, dict_layer in reversed(list(enumerate(dict_network))):
        
        activation = dict_layer['activation']
        activation_params = dict_layer['activation_params']
        Z = dict_layer['Z']
        X = dict_layer['X']
        V = dict_layer['V']
        W = dict_layer['W']
        
        if len(activation_params) == 0:
            raise Exception('Activation parameter is empty.')      
            
        elif activation == 'logi':
            a = activation_params[0]
            dZ =  a * error * Z * (1-Z)
            
        elif activation == 'tanh':
            a = activation_params[0]
            c = activation_params[1]
            dZ = (c / a) * error * (a - Z) * (a + Z)
            
        elif activation == 'relu':
            a = activation_params[0]
            mult = np.where(V > 0, 1, a)
            dZ = mult * error
            
        else:
            raise Exception('Activation function is not supported')
        
        dict_layer['dZ'] = dZ
        dW = learning_rate * np.outer(dZ, X)
        dict_layer['dW'] = dW
        dB = learning_rate * dZ
        dict_layer['dB'] = dB
        error =  np.dot(dZ, W) 
        
    return dict_network, error

In [5]:
def update_weights(init_dict_network, update_dict_network, delta_weights, delta_biases, momentum):
    
    for index, dict_layer in enumerate(update_dict_network):
        
        curr_weight = dict_layer['weight']
        curr_bias = dict_layer['bias']
        
        prev_delta_weight = dict_layer['prev_delta_weight']
        prev_delta_bias = dict_layer['prev_delta_bias']
        
        delta_weight = np.array(delta_weights[index])
        delta_bias = np.array(delta_biases[index])
        delta_bias = delta_bias.reshape(-1,1)

        update_bias = curr_bias + momentum * prev_delta_bias + delta_bias
        update_weight = curr_weight + momentum * prev_delta_weight + delta_weight

        init_dict_network[index]['weight'] = update_weight
        init_dict_network[index]['bias'] = update_bias
        
        init_dict_network[index]['prev_delta_weight'] = delta_weight
        init_dict_network[index]['prev_delta_bias'] = delta_bias
        
    return init_dict_network

In [6]:
'''
!!!YOU CAN INPUT ANY NUMBER OF NODES AND LAYERS HERE!!!
!!!   JUST ADD A DICTIONARY FOR A LAYER SPECIFICS   !!!
!!!   CHANGE DIM_IN FOR NO OF NODES IN PREV LAYER   !!!
!!!   CHANGE DIM_OUT FOR NO OF NODES IN CUR LAYER   !!!
'''

input1 = [0.4, 0.5, -0.7]
output1 = [0.83, 0.74]

dict_network_tanh = [
    {'dim_in': 3, 'dim_out': 4, 'activation': 'tanh', 'activation_params': [1.716, 2/3]},
    {'dim_in': 4, 'dim_out': 3, 'activation': 'tanh', 'activation_params': [1.716, 2/3]},
    {'dim_in': 3, 'dim_out': 2, 'activation': 'tanh', 'activation_params': [1.716, 2/3]}
]

dict_network_relu = [
    {'dim_in': 3, 'dim_out': 4, 'activation': 'relu', 'activation_params': [0.01]},
    {'dim_in': 4, 'dim_out': 3, 'activation': 'relu', 'activation_params': [0.01]},
    {'dim_in': 3, 'dim_out': 2, 'activation': 'relu', 'activation_params': [0.01]}
]

dict_network_logi = [
    {'dim_in': 3, 'dim_out': 4, 'activation': 'logi', 'activation_params': [0.7]},
    {'dim_in': 4, 'dim_out': 3, 'activation': 'logi', 'activation_params': [0.7]},
    {'dim_in': 3, 'dim_out': 2, 'activation': 'logi', 'activation_params': [0.7]}
]

In [7]:
dict_network = dict_network_relu
dict_network = initialize(dict_network)
dict_network = forward_propagation(input1, dict_network)
dict_network, error = backward_propagation(output1, dict_network)
print(error)

[-8.24147125e-05  1.51269444e-05 -1.81116624e-05]


In [8]:
print('NETWORK ARCHITECTURE')
i = 0
for dict_layer in dict_network:
    i += 1
    print(f"\nLAYER {i}")
    print(f"\nInput: \n{dict_layer['X']}")
    print(f"Init Weight: \n{dict_layer['W']}")
    print(f"Init Bias: \n{dict_layer['B']}")
    print(f"V: \n{dict_layer['V']}")
    print(f"Output: \n{dict_layer['Z']}\n")
    
    print(f"Delta: \n{dict_layer['dZ']}")
    print(f"New Weight: \n{dict_layer['dW']}")
    print(f"New Bias: \n{dict_layer['dB']}")
    

NETWORK ARCHITECTURE

LAYER 1

Input: 
[0.4, 0.5, -0.7]
Init Weight: 
[[ 0.34455712 -0.0238432   0.65099035]
 [-0.93710722  0.61609993  0.13123484]
 [-0.404755   -0.90660856  0.9812548 ]
 [-0.98634853  0.53958606  0.4935342 ]]
Init Bias: 
[[-0.24512213]
 [-0.0117051 ]
 [ 0.85789678]
 [-0.20909191]]
V: 
[-0.57491412 -0.17036241 -0.44418786 -0.67931224]
Output: 
[-0.00574914 -0.00170362 -0.00444188 -0.00679312]

Delta: 
[-7.66078949e-05  3.09663281e-05  1.82162721e-05  1.98986660e-05]
New Weight: 
[[-2.60466843e-05 -3.25583553e-05  4.55816975e-05]
 [ 1.05285516e-05  1.31606894e-05 -1.84249652e-05]
 [ 6.19353252e-06  7.74191565e-06 -1.08386819e-05]
 [ 6.76554643e-06  8.45693303e-06 -1.18397062e-05]]
New Bias: 
[-6.51167107e-05  2.63213789e-05  1.54838313e-05  1.69138661e-05]

LAYER 2

Input: 
[-0.00574914 -0.00170362 -0.00444188 -0.00679312]
Init Weight: 
[[ 0.94791259  0.04882943 -0.81277381  0.62661683]
 [-0.57662643  0.10869157 -0.41546177  0.63228472]
 [ 0.65608513 -0.55684526  0.2896